**Proyecto final Python**

*   **Nombre: Alexa Fernández Gavilano**
*   **Código: 20216513**

In [95]:
!pip install wbgapi


In [96]:
import pandas as pd
import altair as alt
import wbgapi as wb

In [97]:
from google.colab import files
from google.colab import drive

drive.mount('/content/drive')

ruta = "/content/drive/MyDrive/Proyectofinal/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
df = pd.read_excel("wdh2024.xls", sheet_name="Sheet1")

print(df.columns.tolist())

['Country name', 'Ladder score', 'upperwhisker', 'lowerwhisker', 'Explained by: Log GDP per capita', 'Explained by: Social support', 'Explained by: Healthy life expectancy', 'Explained by: Freedom to make life choices', 'Explained by: Generosity', 'Explained by: Perceptions of corruption', 'Dystopia + residual']


In [99]:
df_clean = df.dropna()
print(f'Datos originales: {df.shape}')
print(f'Datos limpios (sin NINGÚN NaN): {df_clean.shape}')
print(f'NaN eliminados: {df.shape[0] - df_clean.shape[0]} filas')

Datos originales: (143, 11)
Datos limpios (sin NINGÚN NaN): (140, 11)
NaN eliminados: 3 filas


In [100]:
df_clean

,Country name,Ladder score,upperwhisker,lowerwhisker,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,7.7407,7.814808,7.666591,1.844054,1.572367,0.694793,0.859294,0.141674,0.546158,2.082367
1,Denmark,7.5827,7.665297,7.500103,1.907798,1.520361,0.698944,0.822740,0.203579,0.548413,1.880887
2,Iceland,7.5251,7.617527,7.432673,1.880718,1.616525,0.718345,0.818509,0.258344,0.182473,2.050233
3,Sweden,7.3441,7.421563,7.266637,1.878146,1.500826,0.723871,0.838269,0.221455,0.523795,1.657707
4,Israel,7.3411,7.405165,7.277036,1.802934,1.512793,0.739816,0.641463,0.153205,0.192823,2.298026
...,...,...,...,...,...,...,...,...,...,...,...
138,Congo (Kinshasa),3.2954,3.462328,3.128472,0.533670,0.664700,0.261508,0.472705,0.188676,0.072167,1.102014
139,Sierra Leone,3.2452,3.366252,3.124148,0.654278,0.565977,0.253321,0.469035,0.181365,0.053299,1.067906
140,Lesotho,3.1862,3.468785,2.903615,0.770807,0.850587,0.000000,0.523188,0.082087,0.084969,0.874525
141,Lebanon,2.7065,2.796744,2.616256,1.376726,0.576713,0.556247,0.173223,0.067641,0.028772,-0.072831


In [101]:
# REQUISITO: Función que establecemos con 4 niveles (Alta, Media Alta, Media y Baja)
def categorizar_salud_v2(valor):
    """
    Clasifica el impacto de la salud en 4 niveles basados en los datos de 2024.
    """
    if valor >= 0.70:
        return '4. Expectativa de salud alta'
    elif valor >= 0.55:
        return '3. Expectativa de salud media alta'
    elif valor >= 0.35:
        return '2. Expectativa de salud media'
    else:
        return '1. Expectativa de salud baja'

# Aplicamos la transformación creando una nueva columna
df_happy['Categoria_Salud'] = df_happy['Explained by: Healthy life expectancy'].apply(categorizar_salud_v2)

In [102]:
# Hacemos la transformación de las columnas, porque me salía error sin hacer este paso
nombre_columna_original = 'Explained by: Healthy life expectancy'
df_happy['Categoria_Salud'] = df_happy[nombre_columna_original].apply(categorizar_salud_v2)

# Manejamos la paleta de colores de forma simple para que el gráfico sea atractivo
orden_niveles = ['4. Expectativa de salud alta', '3. Expectativa de salud media alta', '2. Expectativa de salud media', '1. Expectativa de salud baja']
paleta_colores = ['#1a9850', '#a6d96a', '#fdae61', '#d73027'] # De verde (bueno) a rojo (malo)

In [103]:
# Insertamos el código del gráfico
grafico_limpio = alt.Chart(df_happy).mark_bar().encode(
    x=alt.X('count():Q', title='Cantidad de Países'),
    y=alt.Y('Categoria_Salud:N',
            sort=orden_niveles,
            title='Distribución de países por categorías',
            # Para efectos estéticos, eliminé la etiqueta, porque ya hay leyenda
            axis=alt.Axis(labels=False, ticks=False)),
    color=alt.Color('Categoria_Salud:N',
                    scale=alt.Scale(domain=orden_niveles, range=paleta_colores),
                    legend=alt.Legend(title="Categorías de Salud")),
    tooltip=[alt.Tooltip('count()', title='Total de Países'),
             alt.Tooltip('Categoria_Salud', title='Categoría')]
).properties(
    width=600,
    height=250,
    title="Distribución Global: Expectativa de Vida Saludable (2024)"
)

grafico_limpio.display()

alt.Chart(...)

#**Brechas de Bienestar entre el Norte y el Sur Global**

Para entender la distribución de la felicidad a nivel planetario, es imperativo fragmentar los datos bajo una lógica geopolítica. En esta sección, utilizamos un gráfico de barras concatenado verticalmente para visualizar el Ladder Score (Puntaje de Felicidad). Esta elección técnica no es al azar: al separar los dataframes y usar escalas independientes, logramos comparar la jerarquía interna de cada bloque sin que las diferencias de magnitud oculten las realidades locales. Podríamos poner a prueba la hipótesis de que en el Norte Global exhibe una homogeneidad en puntajes altos, el Sur Global revela una heterogeneidad que refleja sus diversas transiciones económicas y sociales.

In [104]:
# Para esta parte usamos listas amplias y una forma académica de estudiarlo: Sur global vs Norte global
#Norte global: Europa, EEUU y Canadá
#Sur global:Latinamérica, Asia y África Subsahariana
norte_global = ['Finland', 'Denmark', 'Iceland', 'Norway', 'Sweden', 'Switzerland',
    'Netherlands', 'Canada', 'United States', 'Germany', 'France',
    'United Kingdom', 'Belgium', 'Austria', 'Luxembourg']

sur_global = ['Argentina', 'Chile', 'Uruguay', 'Brazil', 'Peru', 'Colombia', 'Mexico',
    'India', 'Pakistan', 'Philippines', 'Vietnam', 'Thailand','Nigeria', 'Ethiopia', 'South Africa', 'Kenya', 'Zimbabwe', 'Botswana']

# REQUISITO: Manejo de DataFrames independientes
df_norte = df_happy[df_happy['Country name'].isin(norte_global)].copy()
df_sur = df_happy[df_happy['Country name'].isin(sur_global)].copy()

In [105]:
# Gráfico para el Norte Global
chart_norte = alt.Chart(df_norte).mark_bar().encode(
    x=alt.X('Ladder score:Q', title='Puntaje de Felicidad'),
    y=alt.Y('Country name:N', sort='-x', title='Países Norte Global'),
    color=alt.value('#4c78a8'), # Azul
    tooltip=['Country name', 'Ladder score']
).properties(title='Felicidad en el Norte Global', width=400, height=200)

# Gráfico para el Sur Global
chart_sur = alt.Chart(df_sur).mark_bar().encode(
    x=alt.X('Ladder score:Q', title='Puntaje de Felicidad'),
    y=alt.Y('Country name:N', sort='-x', title='Países Sur Global'),
    color=alt.value('#e15759'), # Rojo
    tooltip=['Country name', 'Ladder score']
).properties(title='Felicidad en el Sur Global', width=400, height=200)

# REQUISITO: Concatenación como en los notebooks de la Clase 4
# Esto elimina los espacios vacíos y pone un gráfico debajo del otro
grafico_final_barras = alt.vconcat(chart_norte, chart_sur).resolve_scale(y='independent')

grafico_final_barras

alt.VConcatChart(...)

#**PBI vs Salud**
Gráfico extra: La elección de un gráfico de dispersión para analizar la relación entre el PIB per cápita y la expectativa de vida saludable responde a la necesidad de identificar patrones de correlación estructural en el desarrollo humano. Mientras que las barras nos permiten observar magnitudes aisladas, la dispersión nos revela la eficiencia de los sistemas nacionales: nos permite visualizar si mayores niveles de riqueza económica se traducen linealmente en una mejor salud pública o si existen anomalías (países que, con menos recursos, logran resultados sanitarios superiores). Al incorporar nuestra categorización de soporte social mediante el canal del color, el gráfico deja de ser una simple nube de puntos para convertirse en una herramienta de análisis multidimensional que expone las brechas de desigualdad entre el Norte y el Sur Global.

In [114]:
# GRÁFICO: Concatenación Horizontal (Dispersión PIB vs Salud)

# Gráfico para el Norte Global
# Usamos \: para que Altair no confunda los ":" del nombre con tipos de datos
scatter_norte = alt.Chart(df_norte).mark_circle(size=100).encode(
    x=alt.X('Explained by\\: Log GDP per capita:Q', title='Impacto PIB'),
    y=alt.Y('Explained by\\: Healthy life expectancy:Q', title='Expectativa Salud'),
    color=alt.Color('Nivel_Soporte:N', legend=alt.Legend(title="Soporte Social")),
    tooltip=['Country name']
).properties(
    title='Norte Global',
    width=280,
    height=280
)

# Gráfico para el Sur Global
scatter_sur = alt.Chart(df_sur).mark_circle(size=100).encode(
    x=alt.X('Explained by\\: Log GDP per capita:Q', title='Impacto PIB'),
    y=alt.Y('Explained by\\: Healthy life expectancy:Q', title='Expectativa Salud'),
    color=alt.Color('Nivel_Soporte:N'),
    tooltip=['Country name']
).properties(
    title='Sur Global',
    width=280,
    height=280
)

#Concatenación horizontal con escalas de color compartidas
grafico_final_scatter = alt.hconcat(scatter_norte, scatter_sur).resolve_scale(color='shared')

grafico_final_scatter.display()

alt.HConcatChart(...)